In [1]:
from imports import *

/Users/jniedziela/.pyenv/versions/3.11.4/envs/master/lib/python3.11/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
joints = [
    'right_shoulder', 'right_elbow', 'right_wrist', 
    'nose', 
    'left_shoulder', 'left_elbow', 'left_wrist', 
    'right_hip', 'right_knee', 'right_ankle', 
    'left_hip', 'left_knee', 'left_ankle'
    ]

phase_colors = {'run-up': 'lightblue', 'jump': 'lightgreen', 'attack': 'lightpink', 'landing': 'wheat'}

EXCEL_DATA_DIR = '../bpe/attack_pose_data/DATASETS/'

# 4  phases (3 breakpoitns)

Divide the series into four splits, and then compare each.

In [4]:
PLOT = False
SAVE_PLOTS = True
PLOTS_DIR = '../bpe/attack_pose_data/plots/'

sep_points_num = []
sep_points_all = []
sep_points_all_attack_normalised = []
analysis_dict = {
    'file':[],
    'phase':[],
    'num_breakpoints':[],
    'breakpoints':[],
    'breakpoints_reindexed':[],
    'breakpoints_attack_normalised':[],
    'y_pca_signal':[],
    'x_pca_signal':[],
    **{joint+'_x':[] for joint in joints},
    **{joint+'_y':[] for joint in joints}
    }

if PLOT:
    fig, ax = plt.subplots(len(os.listdir(EXCEL_DATA_DIR)[:20]), 1, figsize=(15, 70))
    files = os.listdir(EXCEL_DATA_DIR)[:20]
else:
    files = os.listdir(EXCEL_DATA_DIR)

for i, filename in enumerate(files):

    if SAVE_PLOTS:
        fig2, ax2 = plt.subplots(1, 1, figsize=(15, 10))

    # Load
    df1 = pd.read_excel(EXCEL_DATA_DIR + filename, index_col=0)
    df1.rename(columns=MAPPINGS, inplace=True)

    # Save the phase column -- as the index will be reset

    # For the REFERENCE_POSE_table, we don't want to aggregate the numeric columns -- fps was 24
    if filename != 'Techniek_aanval_met_sprong_Ref_POSE_tabel.xlsx':
        fps = df1.fps.values[0]
        phase_col = aggregate_phases(df1, fps, 24)
        df1, index = aggregate_all_numeric_columns(df1, fps, 24)
        index = index[0]
    else:
        phase_col = df1['phase']
        index = {i:i for i in range(len(df1))}

    # break
    norm_df = bbox_normalize_joint_coordinates(df1[[joint + '_x' for joint in joints] + [joint + '_y' for joint in joints]].copy())

    # Normalise the keypoints
    normalised_keypoints_y = np.array([norm_df[joint + '_y_normalized'].values for joint in joints])
    normalised_keypoints_x = np.array([norm_df[joint + '_x_normalized'].values for joint in joints])

    # PCA on y values of normalised keypoints
    pca = PCA(n_components=1)
    Y = pca.fit_transform(normalised_keypoints_y.T)
    if PLOT:
        ax[i].plot(Y, label=filename, color='r')
    if SAVE_PLOTS:
        ax2.plot(Y, label=filename, color='r')

    pca = PCA(n_components=1)
    X = pca.fit_transform(normalised_keypoints_x.T)
    if PLOT:
        ax[i].plot(X, color='g')
    if SAVE_PLOTS:
        ax2.plot(X, color='g')

    # Concatenate the x and y values -- more features for the rupture detection algorithm
    normalised_keypoints = np.concatenate((normalised_keypoints_x, normalised_keypoints_y), axis=0)

    # Detect the breakpoints
    algo = rpt.KernelCPD(kernel="rbf", min_size=5)
    algo.fit(normalised_keypoints.T)
    # result = algo.predict(pen=pen)
    result = algo.predict(n_bkps=3)
    result_reindexed = []
    for bkp in result:
        for k,v in index.items():
            if bkp == v:
                result_reindexed.append(k)
                break
    if len(result) != len(result_reindexed):
        result_reindexed.append(list(index.keys())[-1])

    # Plot the breakpoints
    if PLOT:
        for bkp in result:
            ax[i].axvline(bkp, color='r')

    if SAVE_PLOTS:
        for bkp in result:
            ax2.axvline(bkp, color='r')

    # Save the breakpoints
    attack = phase_col[(phase_col == 'attack')].index[0]
    if PLOT:
        ax[i].axvline(attack, color='g')
    if SAVE_PLOTS:
        ax2.axvline(attack, color='g')
    sep_points_num.append(len(result))
    sep_points_all.extend(result[:-1])
    sep_points_all_attack_normalised.extend([bkp - attack for bkp in result][:-1])

    # Plot the phases
    if PLOT:
        start_idx = 0
        current_phase = phase_col[0]
        for idx, phase in enumerate(phase_col):
            if phase != current_phase or idx == len(phase_col) - 1:
                end_idx = idx if phase != current_phase else len(phase_col)
                ax[i].fill_betweenx(ax[i].get_ylim(), start_idx, end_idx, color=phase_colors[current_phase], alpha=0.3)
                start_idx = idx
                current_phase = phase

        # Plot labels
        if len(result) != 4:
            print(filename, len(result))
            # ax[i].set_title(filename, color='r')
            ax[i].legend(loc='upper left', labelcolor='r')
        else:
            ax[i].legend(loc='upper left', labelcolor='g')
    
    if SAVE_PLOTS:
        start_idx = 0
        current_phase = phase_col[0]
        for idx, phase in enumerate(phase_col):
            if phase != current_phase or idx == len(phase_col) - 1:
                end_idx = idx if phase != current_phase else len(phase_col)
                ax2.fill_betweenx(ax2.get_ylim(), start_idx, end_idx, color=phase_colors[current_phase], alpha=0.3)
                start_idx = idx
                current_phase = phase

        # Plot labels
        if len(result) != 4:
            print(filename, len(result))
            # ax[i].set_title(filename, color='r')
            ax2.legend(loc='upper left', labelcolor='r')
        else:
            ax2.legend(loc='upper left', labelcolor='g')
        ax2.set_title(filename)

    if SAVE_PLOTS:
        fig2.savefig(PLOTS_DIR + filename[:-5] + '_breakpoints.png')
        plt.close(fig2)

    # Save the analysis
    analysis_dict['file'].extend([filename] * len(phase_col))
    analysis_dict['phase'].extend(phase_col.values)
    analysis_dict['num_breakpoints'].extend([len(result) - 1] * len(phase_col))
    analysis_dict['breakpoints'].extend([result] * len(phase_col))
    analysis_dict['breakpoints_reindexed'].extend([result_reindexed] * len(phase_col))
    analysis_dict['breakpoints_attack_normalised'].extend([[bkp - attack for bkp in result][:-1]] * len(phase_col))
    analysis_dict['y_pca_signal'].extend(Y.reshape(-1))
    analysis_dict['x_pca_signal'].extend(X.reshape(-1))
    for i, joint in enumerate(joints):
        analysis_dict[joint+'_x'].extend(normalised_keypoints_x[i])
        analysis_dict[joint+'_y'].extend(normalised_keypoints_y[i])

if PLOT:
    fig.tight_layout()

In [32]:
analysis_df = pd.DataFrame(analysis_dict)
analysis_df

,file,phase,num_breakpoints,breakpoints,breakpoints_reindexed,breakpoints_attack_normalised,y_pca_signal,x_pca_signal,right_shoulder_x,right_elbow_x,...,nose_y,left_shoulder_y,left_elbow_y,left_wrist_y,right_hip_y,right_knee_y,right_ankle_y,left_hip_y,left_knee_y,left_ankle_y
0,GX010044_Clip_14_POSE_tabel.xlsx,run-up,3,"[29, 47, 64, 89]","[58, 94, 128, 176]","[-26.0, -8.0, 9.0]",-0.570667,2.162022,0.005500,0.005272,...,0.074818,0.135325,0.097390,0.111748,0.220890,0.196629,0.031614,0.216495,0.199683,0.070299
1,GX010044_Clip_14_POSE_tabel.xlsx,run-up,3,"[29, 47, 64, 89]","[58, 94, 128, 176]","[-26.0, -8.0, 9.0]",-0.568334,2.170478,0.004583,0.002433,...,0.065693,0.135325,0.099398,0.114613,0.217466,0.204120,0.036606,0.214777,0.201268,0.080844
2,GX010044_Clip_14_POSE_tabel.xlsx,run-up,3,"[29, 47, 64, 89]","[58, 94, 128, 176]","[-26.0, -8.0, 9.0]",-0.567462,2.177807,0.002291,0.000406,...,0.054745,0.135325,0.102410,0.116762,0.215753,0.211610,0.041597,0.214777,0.204437,0.079086
3,GX010044_Clip_14_POSE_tabel.xlsx,run-up,3,"[29, 47, 64, 89]","[58, 94, 128, 176]","[-26.0, -8.0, 9.0]",-0.567597,2.181372,0.000917,0.000000,...,0.052920,0.130053,0.104418,0.118195,0.212329,0.211610,0.046589,0.211340,0.210777,0.082601
4,GX010044_Clip_14_POSE_tabel.xlsx,run-up,3,"[29, 47, 64, 89]","[58, 94, 128, 176]","[-26.0, -8.0, 9.0]",-0.562941,2.182615,0.000000,0.000000,...,0.049270,0.135325,0.104418,0.118911,0.214041,0.213483,0.048253,0.216495,0.213946,0.082601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,GX010045_Clip_0_POSE_tabel.xlsx,landing,3,"[24, 43, 59, 89]","[48, 86, 118, 176]","[-29.0, -10.0, 6.0]",0.493622,-0.887277,0.897874,0.833134,...,0.495868,0.489642,0.311633,0.290715,0.516667,0.544484,0.601527,0.514680,0.502950,0.367580
4028,GX010045_Clip_0_POSE_tabel.xlsx,landing,3,"[24, 43, 59, 89]","[48, 86, 118, 176]","[-29.0, -10.0, 6.0]",0.374525,-0.869484,0.889497,0.829844,...,0.431818,0.436911,0.289221,0.269406,0.477778,0.508897,0.592366,0.481865,0.480826,0.364155
4029,GX010045_Clip_0_POSE_tabel.xlsx,landing,3,"[24, 43, 59, 89]","[48, 86, 118, 176]","[-29.0, -10.0, 6.0]",0.219022,-0.857614,0.884987,0.827751,...,0.376033,0.378531,0.261473,0.248097,0.407407,0.469751,0.560305,0.412781,0.443953,0.356164
4030,GX010045_Clip_0_POSE_tabel.xlsx,landing,3,"[24, 43, 59, 89]","[48, 86, 118, 176]","[-29.0, -10.0, 6.0]",0.080616,-0.842779,0.883054,0.821172,...,0.326446,0.325800,0.223052,0.226788,0.353704,0.428826,0.520611,0.362694,0.418879,0.350457


In [33]:
reference_df = analysis_df[analysis_df['file'] == 'REFERENCE_POSE_table.xlsx'].reset_index(drop=True)

reference_breakpoints = reference_df['breakpoints'].values[0]

print(reference_breakpoints)
reference_df

[39, 52, 67, 91]


,file,phase,num_breakpoints,breakpoints,breakpoints_reindexed,breakpoints_attack_normalised,y_pca_signal,x_pca_signal,right_shoulder_x,right_elbow_x,...,nose_y,left_shoulder_y,left_elbow_y,left_wrist_y,right_hip_y,right_knee_y,right_ankle_y,left_hip_y,left_knee_y,left_ankle_y
0,REFERENCE_POSE_table.xlsx,run-up,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",-0.510918,-1.756522,0.000000,0.001961,...,0.304348,0.228814,0.182353,0.097222,0.091667,0.092308,-0.000000,0.089286,0.135593,0.036036
1,REFERENCE_POSE_table.xlsx,run-up,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",-0.506774,-1.758276,0.000000,0.001961,...,0.304348,0.237288,0.188235,0.097222,0.091667,0.100000,-0.000000,0.089286,0.127119,0.036036
2,REFERENCE_POSE_table.xlsx,run-up,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",-0.508112,-1.760039,0.001972,0.001961,...,0.304348,0.228814,0.182353,0.097222,0.091667,0.100000,-0.000000,0.098214,0.127119,0.036036
3,REFERENCE_POSE_table.xlsx,run-up,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",-0.510972,-1.758347,0.001972,0.001961,...,0.297101,0.228814,0.182353,0.092593,0.091667,0.100000,-0.000000,0.098214,0.135593,0.036036
4,REFERENCE_POSE_table.xlsx,run-up,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",-0.524334,-1.759968,0.001972,0.001961,...,0.297101,0.228814,0.176471,0.087963,0.091667,0.092308,-0.000000,0.089286,0.127119,0.027027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,REFERENCE_POSE_table.xlsx,landing,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",0.127068,1.731663,0.984221,0.982353,...,0.434783,0.415254,0.335294,0.162037,0.333333,0.338462,0.245763,0.348214,0.305085,0.252252
87,REFERENCE_POSE_table.xlsx,landing,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",0.093228,1.730682,0.982249,0.976471,...,0.434783,0.406780,0.282353,0.148148,0.316667,0.338462,0.245763,0.330357,0.296610,0.252252
88,REFERENCE_POSE_table.xlsx,landing,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",0.111620,1.715880,0.982249,0.976471,...,0.442029,0.406780,0.311765,0.162037,0.316667,0.330769,0.254237,0.330357,0.296610,0.261261
89,REFERENCE_POSE_table.xlsx,landing,3,"[39, 52, 67, 91]","[39, 52, 67, 90]","[-20, -7, 8]",0.124806,1.707321,0.982249,0.974510,...,0.449275,0.415254,0.317647,0.157407,0.316667,0.330769,0.262712,0.330357,0.296610,0.270270


In [34]:
scores_df = pd.read_excel(DATA_DIR + 'pose_scores.xlsx', index_col=0)
scores_df.loc[len(scores_df), 'pose_tabel'] = 'REFERENCE_POSE_table.xlsx'
scores_df.loc[scores_df.pose_tabel == 'REFERENCE_POSE_table.xlsx', 'handigheid'] = 'RECHTSHANDIG'

for filename in analysis_df.file.unique():
    bkpts = analysis_df[analysis_df.file == filename]['breakpoints_reindexed'].values[0]
    scores_df.loc[scores_df['pose_tabel'] == filename, 'algorithm_detected_breakpoints'] = str(bkpts)

print(scores_df.shape)
scores_df.head()

(44, 16)


,pose_tabel,handigheid,aanloop_aantal_passen,aanloop_aantal_passen_score,aanloop_passen,sprong_pas,rem_pas,rem_pas_score,sprong_hoek,sprong_hoek_score,positie_tov_bal,positie_tov_bal_score,slagarm_hoek,slagarm_hoek_score,totaal_score,algorithm_detected_breakpoints
0,GX010032_Clip_10_sec_24_POSE_tabel.xlsx,LINKSHANDIG,4.0,1.0,['Rechterpas' 'Linkerpas'],Rechterpas,Linkerpas,0.0,55.0,0.388889,[nan 'aanval onder de bal'],0.0,168.0,0.933333,2.322222,"[86, 130, 160, 182]"
1,GX010033_Clip_6_sec_14_POSE_tabel.xlsx,RECHTSHANDIG,3.0,1.0,['Rechterpas'],Linkerpas,Rechterpas,0.0,28.0,0.688889,[nan],1.0,160.0,0.888889,3.577778,"[54, 92, 124, 182]"
2,GX010034_Clip_5_sec_15_POSE_tabel.xlsx,LINKSHANDIG,13.0,1.0,['Rechterpas' 'Linkerpas' 'Rechterpas' 'Linker...,Linkerpas,Rechterpas,1.0,45.0,0.500000,[nan 'aanval onder de bal'],0.0,171.0,0.950000,3.450000,"[54, 90, 122, 182]"
3,GX010036_Clip_4_sec_16_POSE_tabel.xlsx,RECHTSHANDIG,3.0,1.0,['Linkerpas'],Rechterpas,Linkerpas,1.0,48.0,0.466667,[nan 'aanval onder de bal'],0.0,159.0,0.883333,3.350000,"[54, 94, 122, 182]"
4,GX010037_Clip_7_sec_15_POSE_tabel.xlsx,RECHTSHANDIG,5.0,1.0,['Rechterpas' 'Linkerpas' 'Rechterpas'],Linkerpas,Rechterpas,0.0,48.0,0.466667,[nan],1.0,153.0,0.850000,3.316667,"[58, 96, 126, 182]"


In [35]:
ra_cols = ['right_shoulder_x', 'right_shoulder_y', 'right_elbow_x', 'right_elbow_y', 'right_wrist_x', 'right_wrist_y']
la_cols = ['left_shoulder_x', 'left_shoulder_y', 'left_elbow_x', 'left_elbow_y', 'left_wrist_x', 'left_wrist_y']
rl_cols = ['right_hip_x', 'right_hip_y', 'right_knee_x', 'right_knee_y', 'right_ankle_x', 'right_ankle_y']
ll_cols = ['left_hip_x', 'left_hip_y', 'left_knee_x', 'left_knee_y', 'left_ankle_x', 'left_ankle_y']


columns_to_compare = reference_df.columns[6:]
for filename in scores_df[scores_df['handigheid'] == 'RECHTSHANDIG']['pose_tabel'].tolist():

    compared_df = analysis_df[analysis_df['file'] == filename].reset_index(drop=True)

    compared_breakpoints = compared_df['breakpoints'].values[0]

    start_reference = 0
    start_compared = 0

    for i, bkp in enumerate(reference_breakpoints):

        end_reference = reference_breakpoints[i]
        end_compared = compared_breakpoints[i]

        reference_phase_df = reference_df.iloc[start_reference:end_reference].reset_index(drop=True)
        compared_phase_df = compared_df.iloc[start_compared:end_compared].reset_index(drop=True)
        
        for col in columns_to_compare:
            dtw_score = dtw(reference_phase_df[col], compared_phase_df[col])
            scores_df.loc[scores_df['pose_tabel'] == filename, col+f'_score_PHASE_{i}'] = dtw_score

        scores_df.loc[scores_df['pose_tabel'] == filename, f'total_dtw_score_PHASE_{i}'] = dtw(reference_phase_df[columns_to_compare[2:]], compared_phase_df[columns_to_compare[2:]])
        scores_df.loc[scores_df['pose_tabel'] == filename, f'ra_score_PHASE_{i}'] = dtw(reference_phase_df[ra_cols], compared_phase_df[ra_cols])
        scores_df.loc[scores_df['pose_tabel'] == filename, f'la_score_PHASE_{i}'] = dtw(reference_phase_df[la_cols], compared_phase_df[la_cols])
        scores_df.loc[scores_df['pose_tabel'] == filename, f'rl_score_PHASE_{i}'] = dtw(reference_phase_df[rl_cols], compared_phase_df[rl_cols])
        scores_df.loc[scores_df['pose_tabel'] == filename, f'll_score_PHASE_{i}'] = dtw(reference_phase_df[ll_cols], compared_phase_df[ll_cols])

        
        start_reference = end_reference
        start_compared = end_compared

scores_df.tail()

/var/folders/0z/9gp9vcnj7tb2g040j_v9z31r0000gn/T/ipykernel_4358/315166945.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scores_df.loc[scores_df['pose_tabel'] == filename, f'la_score_PHASE_{i}'] = dtw(reference_phase_df[la_cols], compared_phase_df[la_cols])
/var/folders/0z/9gp9vcnj7tb2g040j_v9z31r0000gn/T/ipykernel_4358/315166945.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scores_df.loc[scores_df['pose_tabel'] == filename, f'rl_score_PHASE_{i}'] = dtw(reference_phase_df[rl_cols], compared_phase_df[rl_cols])
/var/f

,pose_tabel,handigheid,aanloop_aantal_passen,aanloop_aantal_passen_score,aanloop_passen,sprong_pas,rem_pas,rem_pas_score,sprong_hoek,sprong_hoek_score,...,right_knee_y_score_PHASE_3,right_ankle_y_score_PHASE_3,left_hip_y_score_PHASE_3,left_knee_y_score_PHASE_3,left_ankle_y_score_PHASE_3,total_dtw_score_PHASE_3,ra_score_PHASE_3,la_score_PHASE_3,rl_score_PHASE_3,ll_score_PHASE_3
39,GX010047_Clip_0_POSE_tabel.xlsx,RECHTSHANDIG,3.0,1.0,['Linkerpas'],Rechterpas,Linkerpas,1.0,57.0,0.366667,...,1.060990,0.576462,0.667847,0.892613,0.858750,3.025084,1.028432,0.895373,1.598006,1.889467
40,GX010047_Clip_1_POSE_tabel.xlsx,RECHTSHANDIG,5.0,1.0,['Linkerpas' 'Rechterpas' 'Linkerpas'],Rechterpas,Linkerpas,1.0,34.0,0.622222,...,0.352510,0.569610,0.207157,0.191178,0.350342,2.680891,1.394490,1.282336,1.166531,1.035526
41,GX010047_Clip_2_POSE_tabel.xlsx,RECHTSHANDIG,5.0,1.0,['Linkerpas' 'Rechterpas' 'Linkerpas'],Rechterpas,Linkerpas,1.0,35.0,0.611111,...,0.403476,0.752230,0.231501,0.167590,0.336224,4.627834,2.283737,2.210808,2.101728,2.114200
42,GX010047_Clip_4_POSE_tabel.xlsx,RECHTSHANDIG,5.0,1.0,['Linkerpas' 'Rechterpas' 'Linkerpas'],Rechterpas,Linkerpas,1.0,43.0,0.522222,...,0.958383,0.778415,0.836296,0.793994,0.677592,2.864454,0.960740,1.059515,1.544929,1.599529
43,REFERENCE_POSE_table.xlsx,RECHTSHANDIG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
scores_df.to_excel(DATA_DIR + 'POSE_SCORES_incl_BREAKPOINTS.xlsx')